In [1]:
import cv2
import numpy as np
import lmdb
import sys,os
import caffe
import random

In [2]:
def load_txt(txt, shuffle):  
    if txt == None:  
        print "txtpath!!!"  
        exit(0)  
    if not os.path.exists(txt):  
         print "the txt is't exists"  
         exit(0)  
    flag = 0  
    file_content = []  
    txt_file = open(txt, 'r')  
    for line in open(txt, 'r'):  
        line = txt_file.readline()  
        list = line.split()  
        file_content.append([list[0],list[1]])  
        flag += 1   
    if shuffle == 1:
        random.shuffle(file_content)  
    return file_content  

## change the path name

In [3]:
adc_train_prefix="/home/lcy/WORKSPACE/CAM/models/Prostate/ADC/ADCTRAIN/"
t2_train_prefix="/home/lcy/WORKSPACE/CAM/models/Prostate/T2/T2TRAIN/"
lmdb_path="/home/lcy/WORKSPACE/CAM/models/Prostate/multimodal_lmdb_train_test"

## change the txt file name

In [4]:
content = []    
content = load_txt("/home/lcy/WORKSPACE/CAM/models/Prostate/trainname.txt", 1)#加载图片名字和label

In [5]:
print 'total: ', len(content)

total:  8337


In [7]:
env = lmdb.Environment(lmdb_path, map_size=int(1e12))

In [8]:
with env.begin(write=True) as txn:  
    # txn is a Transaction object  
    for i in range(len(content)):  
        datum = caffe.proto.caffe_pb2.Datum()  
        adc_path = adc_train_prefix + content[i][0]  
        t2_path = t2_train_prefix + content[i][0]  
        label = int(content[i][1])
        
        adc_image=cv2.imread(adc_path, cv2.IMREAD_GRAYSCALE)
        t2_image=cv2.imread(t2_path, cv2.IMREAD_GRAYSCALE)
        
        adc_image=cv2.resize(adc_image,(256,256))
        t2_image=cv2.resize(t2_image,(256,256))
        
        datum.channels = 2#channels
        datum.height = adc_image.shape[0]#height  
        datum.width = adc_image.shape[1]#width  
        data = np.zeros((2,  adc_image.shape[0],  adc_image.shape[1]), dtype=np.uint8)#初始化data
        
        data[0] = adc_image  
        data[1] = t2_image  
        datum.data = data.tostring() #data  
        datum.label = int(label)#label  
        str_id = '{:08}'.format(i)   #顺序+图片名字作为key
        if i % 100 == 0:
            print str_id + '\n'
        txn.put(str_id, datum.SerializeToString())  

00000000

00000100

00000200

00000300

00000400

00000500

00000600

00000700

00000800

00000900

00001000

00001100

00001200

00001300

00001400

00001500

00001600

00001700

00001800

00001900

00002000

00002100

00002200

00002300

00002400

00002500

00002600

00002700

00002800

00002900

00003000

00003100

00003200

00003300

00003400

00003500

00003600

00003700

00003800

00003900

00004000

00004100

00004200

00004300

00004400

00004500

00004600

00004700

00004800

00004900

00005000

00005100

00005200

00005300

00005400

00005500

00005600

00005700

00005800

00005900

00006000

00006100

00006200

00006300

00006400

00006500

00006600

00006700

00006800

00006900

00007000

00007100

00007200

00007300

00007400

00007500

00007600

00007700

00007800

00007900

00008000

00008100

00008200

00008300



In [11]:
env.close()

# check randomly

In [12]:
env = lmdb.open(lmdb_path, readonly=True)
with env.begin() as txn:
    raw_datum = txn.get(b'00000100')

datum = caffe.proto.caffe_pb2.Datum()
datum.ParseFromString(raw_datum)

flat_x = np.fromstring(datum.data, dtype=np.uint8)
x = flat_x.reshape(datum.channels, datum.height, datum.width)
y = datum.label

In [13]:
print x.shape

(2, 256, 256)


In [14]:
temp_image=np.zeros((256,256,3),dtype='uint8')
temp_image[:,:,0]=x[0]
temp_image[:,:,1]=x[1]

In [15]:
cv2.imwrite("/home/lcy/WORKSPACE/CAM/models/temp_image.png",temp_image)

True